# 2D Flapping Wings Simulation Notebook Overview

*The following notebook is the Python conversion of Prof. Denda's MATLAB code for his Flapping Wings Research*

Instructions:

* Run the code blocks in order up to the 6th Section `Input Variables`
    * The sections after that can be modified for faster runtime
* Numerical Results are saved to `output.pickle` and graphical output is stored in `fig/`

# Setting Up Environment
*Time to get our Google Colab ready!*


We start by granting access to our Google Drive, which will be used to store all files. This may take a while depending on how much data you have in your Google Drive.

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
except:
    print("Google Drive not found.")

Google Drive not found.


Then we will make a `fig` directory which we will use to store the wake and velocity figures

In [3]:
import os
figuresDir = "drive/MyDrive/figures/"

if not os.path.exists(f"{figuresDir}wake/"):
    os.makedirs(f"{figuresDir}wake/")

if not os.path.exists(f"{figuresDir}velocity/"):
    os.makedirs(f"{figuresDir}velocity/")

Now we can install all necessary libraries, packages, and modules

In [6]:
%pip install numpy scipy matplotlib tqdm

import logging
from multiprocessing.pool import ThreadPool
from tqdm import tqdm
from datetime import datetime

import math
import numpy as np
from scipy.linalg import lu_factor, lu_solve
from scipy.interpolate import CubicSpline
from scipy.integrate import quad

%matplotlib inline
import matplotlib.pyplot as plt
plt.ioff()

                                              0.0/77.1 kB ? eta -:--:--
     ---------------------------------------- 77.1/77.1 kB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


All necessary packages should be installed, now lets prep our variables and functions

# Prepping for Runtime

*Colab should be ready, let's get our variables and functions ready*

## Global Variables   

List of global variables to be used in the 2D Simulation:

* **tau**: Phase shift for the time
    * Start from TOP (0), Between: Start with down stroke (0 < tau < 1),
    * Start from BOTTOM (1), Between: Start with up stroke (1 < tau < 2),
    * 0 <= tau < 2

* **mplot**: Airfoil mesh plot: yes (1), no (0), Compare equal arc and equal abscissa mesh pointts (2)

* **vplot**: Airfoil normal velocity plot: yes (1), no (0)

* **eps**: Used for Modified Biot-Savart Equation

* **wplot**: Wake vortex plot: yes (1), no (0)

* **zavoid**: Zavoid: yes (1), no (0)

* **mpath**: Motion path parameter 
    * No tail (0)
    * DUTail; 2 periods (1)
    * UDTail; 2 periods (2)
    * DUDUTail; 4 periods (3)
    * UDUDTail; 4 periods (4)

* **delta**: Distance between the collocation point and the vortex point on the wing

* **ibios**: Vortex core model (Modified Biot-Savart Equation): yes (1), no (0)

* **svCont**: Space-Fixed Velocity Plot

* **wvCont**: Wing-Fixed Velocity Plot

* **ivCont**:          TODO: what description of ivCont goes here -IP
    * Use of svCont and ivCont: yes (1), no (0)
    * The velocity range varies widely depending on the input parameters
    * It is recommended to respecify this when input parameters are changed.

* **vpFreq**: Frequency of the velocity plots

* **vfplot**: TODO -IP

In [7]:
class globalVar:
    tau = 0
    mplot = 1
    vplot = 1
    eps = 0.5e-6
    wplot = 1
    zavoid = 0
    mpath = 0
    delta = 0
    svCont = 0
    wvCont = 0
    ivCont = 0
    vpFreq = 1
    vfplot = 1

## Functions Used In Preprocessing

* Input Data `in_data`
* Mesh Radius `mesh_r`
* Matrix Coefficient`matrix_coef`
* Camber Mesh `c_mesh`, `camber_mesh`

### Input Data

In [8]:
def nd_data(l_, phiT, phiB, c_, x_, y_, a_, U_, V_, T_):
    dT_ = l_ * np.sin(phiT)
    dB_ = l_ * np.sin(-phiB)
    d_ = dT_ + dB_
    e_ = dT_ - dB_
    e = e_ / d_
    c = c_ / d_
    a = a_ / d_
    x = x_ / d_
    y = y_ / d_
    t_ = T_ / 2.0
    v_ = d_ / t_
    U = U_ / v_
    V = V_ / v_

    return v_, t_, d_, e, c, x, y, a, U, V


def in_data(l_,
            phiT_,
            phiB_,
            c_,
            x_,
            y_,
            a_,
            beta_,
            f_,
            gMax_,
            U_,
            V_):
    T_ = 1.0 / f_

    fac = np.pi / 180.0
    phiT = fac * phiT_
    phiB = fac * phiB_
    beta = fac * beta_
    gMax = fac * gMax_

    v_, t_, d_, e, c, x, y, a, U, V = \
        nd_data(l_, phiT, phiB, c_, x_, y_, a_, U_, V_, T_)

    return v_, t_, d_, e, c, x, y, a, beta, gMax, U, V

### Mesh Radius?

In [9]:
def mesh_r(c, x, y, n, m):
    a = 0.5 * c  # half chord length

    f = CubicSpline(x, y)
    df = f.derivative(nu=1)

    s = [0]

    for i in range(n - 1):
        ds = quad(lambda z: np.sqrt(1 + df(z) ** 2), x[i], x[i+1])
        # Get the first value, cross-checked with matlab code for validation.
        s.append(s[i] + ds[0])

    s = np.array(s)

    gcalc = CubicSpline(s, x)
    dS = s[n - 1] / (m - 1)

    xv = np.zeros((m + 4))
    xv[0] = -a
    xv[1] = gcalc(dS * 0.25)
    xv[2] = gcalc(dS * 0.5)

    for i in range(2, m):
        xv[i + 1] = gcalc(dS * (i - 1))

    xv[m + 1] = gcalc(dS * (m - 1 - 0.5))
    xv[m + 2] = gcalc(dS * (m - 1 - 0.25))
    xv[m + 3] = a

    yv = f(xv)

    xc = np.zeros((m + 3))
    xc[0] = gcalc(dS * 0.125)
    xc[1] = gcalc(dS * 0.375)
    xc[2] = gcalc(dS * 0.75)

    for i in range(2, m - 1):
        xc[i + 1] = gcalc(dS * (i - 0.5))

    xc[m] = gcalc(dS * (m - 1 - 0.75))
    xc[m + 1] = gcalc(dS * (m - 1 - 0.375))
    xc[m + 2] = gcalc(dS * (m - 1 - 0.125))

    yc = df(xc)
    dfc = df(xc)

    mNew = m + 4

    return xv, yv, xc, yc, dfc, mNew

### Matrix Coefficient

In [10]:
def matrix_coef(xv, yv, xc, yc, dfc, m):
    denom = np.sqrt(1 + dfc ** 2)
    nx = -dfc / denom
    ny = 1.0 / denom
    nc = nx + 1j * ny

    zeta = xc + 1j * yc
    zeta0 = xv + 1j * yv

    MVN = np.imag((((1.0 / (np.expand_dims(zeta, 0).transpose() - zeta0)))
                   * nc.reshape((nc.size, 1))) / (2.0 * np.pi))
    MVN = np.append(MVN, np.ones(MVN.shape[1])).reshape((m, m))

    return MVN

### Camber Mesh

In [11]:
def c_mesh(c_, d_):

    epsX = 0.15 * c_
    epsY = 0.15 * c_
    dX = 0.3 * c_
    dY = 0.3 * c_
    maxX = 1.0 * d_
    maxY = 1.0 * d_

    # define the renge in the quadrant
    rX = np.arange(epsX, maxX, dX)
    rY = np.arange(epsY, maxY, dY)

    # Total range
    Xrange = [-np.flip(rX), rX]
    Yrange = [-np.flip(rY), rY]

    # Mesh points
    xi, eta = np.meshgrid(Xrange, Yrange)
    ZETA = xi + 1j * eta
    ZETA /= d_

    return ZETA


def camber_mesh(c_, d_, camber):

    dX = 0.2 * c_
    dY = 0.2 * c_
    maxX = 1.0 * d_
    maxY = 1.0 * d_

    x1 = np.linspace(-0.5, 0.5, dX)
    x2 = np.linspace(0.7, maxX, dX)
    x3 = -np.fliplr(x2)
    x = np.append(x3, [x1, x2])
    nx = x.shape[0]
    atmp_ = 0.5
    y1 = camber * (atmp_ ** 2)
    y2 = 0.0 * x2
    y = np.append(y2, [y1, y2])
    nyh = np.floor(nx / 2)

    for i in range(nyh):
        xi[i+nyh, :] = x
        eta[i+nyh, :] = y + (i - 0.5) * dY
        xi[i, :] = x
        eta[i, :] = y - (nyh - i + 0.5) * dY

    ZETA = complex(xi, eta)
    return ZETA / d_

# Time March Functions

* `airfoil_m`
* `wing_global`
* `airfoil_v`
* `velocity_w2`
* `velocity`

## Airfoil M

### Airfoil M helper functions

In [12]:
class mpath:
    def cos_tail_b2(t):
        if t <= 2.0:
            return np.cos(np.pi * t)
        else:
            return 1

    def cos_tail_g2(t, e):
        tB = t % 4
        return mpath.cos_tail_b2(tB) + e
    
    def d_cos_tail_b2(t):
        if t <= 2.0:
            return -np.pi * np.sin(np.pi * t)
        else:
            return 0

    def d_cos_tail_g2(t):
        tB = t % 4
        return mpath.d_cos_tail_b2(tB)
    
    def d_table_s_tail_b2(t, p, rtOff):
        e0 = np.exp(-2.0 * p * (t - (0.0 + rtOff)))
        e1 = np.exp(-2.0 * p * (t - (1.0 + rtOff)))
        e2 = np.exp(-2.0 * p * (t - (2.0 + rtOff)))
        e4 = np.exp(-2.0 * p * (t - (4.0 + rtOff)))
        f0 = 2.0 * p * e0 / (1.0 + e0) ** 2
        f1 = 4.0 * p * e1 / (1.0 + e1) ** 2
        f2 = 2.0 * p * e2 / (1.0 + e2) ** 2
        f4 = 2.0 * p * e4 / (1.0 + e4) ** 2
        return -f0 + f1 - f2 - f4
    
    def d_table_s_tail_g2(t, p, rtOff):
        tB = t % 4
        return mpath.d_table_s_tail_b2(tB + globalVar.tau)
    
    def table_s_tail_b2(t, p, rtOff):
        f0 = 1.0 / (1.0 + np.exp(t - (0.0 + rtOff)))
        f1 = 1.0 / (1.0 + np.exp(t - (1.0 + rtOff)))
        f2 = 1.0 / (1.0 + np.exp(t - (2.0 + rtOff)))
        f4 = 1.0 / (1.0 + np.exp(t - (4.0 + rtOff)))
        return -f0 + f1 - f2 - f4
    
    def table_s_tail_g2(t, p, rtOff):
        tB = t % 4
        return mpath.table_s_tail_b2(tB + globalVar.tau, rtOff)
    
    def cos_up_tail_b2(t):
        if t <= 2.0:
            return -np.cos(np.pi * t)
        else:
            return -1
        
    def cos_up_tail_g2(t, e):
        tB = t % 4
        return mpath.cos_up_tail_b2(tB) + e
    
    def d_cos_up_tail_b2(t):
        if t <= 2.0:
            return np.pi * np.sin(np.pi * t)
        else:
            return 0
        
    def d_cos_up_tail_g2(t):
        tB = t % 4
        return mpath.d_cos_up_tail_b2(tB)
    
    def d_table_up_s_tail_b2(t, p, rtOff):
        e0 = np.exp(-2.0 * p * (t - (0.0 + rtOff)))
        e1 = np.exp(-2.0 * p * (t - (1.0 + rtOff)))
        e2 = np.exp(-2.0 * p * (t - (2.0 + rtOff)))
        e4 = np.exp(-2.0 * p * (t - (4.0 + rtOff)))
        f0 = 2.0 * p * e0 / (1.0 + e0) ** 2
        f1 = 4.0 * p * e1 / (1.0 + e1) ** 2
        f2 = 2.0 * p * e2 / (1.0 + e2) ** 2
        f4 = 2.0 * p * e4 / (1.0 + e4) ** 2
        return -(-f0 + f1 - f2 - f4)
    
    def d_table_up_s_tail_g2(t, p, rtOff):
        tB = t % 4
        return mpath.d_table_up_s_tail_b2(tB + globalVar.tau, p, rtOff)
    
    def table_up_s_tail_b2(t, p, rtOff):
        f0 = 1.0 / (1.0 + np.exp(-2.0 * p * (t - (0.0 + rtOff))))
        f1 = 2.0 / (1.0 + np.exp(-2.0 * p * (t - (1.0 + rtOff))))
        f2 = 2.0 / (1.0 + np.exp(-2.0 * p * (t - (2.0 + rtOff))))
        f4 = 2.0 / (1.0 + np.exp(-2.0 * p * (t - (4.0 + rtOff))))
        return -(-f0 + f1 - f2 - f4)
    
    def table_up_s_tail_g2(t, p, rtOff):
        tB = t % 4
        return mpath.table_up_s_tail_b2(tB + globalVar.tau, p, rtOff)
    
    def cos_up_tail_b(t):
        if t <= 4.0:
            return -np.cos(np.pi * t)
        else:
            return -1
        
    def cos_up_tail_g(t, e):
        tB = t % 8
        return mpath.cos_up_tail_b(tB) + e
    
    def d_cos_up_tail_b(t):
        if t <= 4.0:
            return np.pi * np.sin(np.pi * t)
        else:
            return 0
        
    def d_cos_up_tail_g(t):
        tB = t % 8
        return mpath.d_cos_up_tail_b(tB)
    
    def d_table_up_s_tail_b(t, p, rtOff):
        e0 = np.exp(-2.0 * p * (t - (0.0 + rtOff)))
        e1 = np.exp(-2.0 * p * (t - (1.0 + rtOff)))
        e2 = np.exp(-2.0 * p * (t - (2.0 + rtOff)))
        e3 = np.exp(-2.0 * p * (t - (3.0 + rtOff)))
        e4 = np.exp(-2.0 * p * (t - (4.0 + rtOff)))
        e8 = np.exp(-2.0 * p * (t - (8.0 + rtOff)))
        f0 = 2.0 * p * e0 / (1.0 + e0) ** 2
        f1 = 2.0 * p * e1 / (1.0 + e0) ** 2
        f2 = 2.0 * p * e2 / (1.0 + e0) ** 2
        f3 = 2.0 * p * e3 / (1.0 + e0) ** 2
        f4 = 2.0 * p * e4 / (1.0 + e0) ** 2
        f8 = 2.0 * p * e8 / (1.0 + e0) ** 2
        return f0 - f1 + f2 - f3 + f4 + f8 # TODO
    
    def d_table_up_s_tail_g(t, p, rtOff):
        tB = t % 8
        return mpath.d_table_up_s_tail_b(tB + globalVar.tau, p, rtOff)
    
    def table_up_s_tail_b(t, p, rtOff):
        f0 = 1.0 / (1.0 + np.exp(-2.0 * p * (0.0 + rtOff)))
        f1 = 1.0 / (1.0 + np.exp(-2.0 * p * (1.0 + rtOff)))
        f2 = 1.0 / (1.0 + np.exp(-2.0 * p * (2.0 + rtOff)))
        f3 = 1.0 / (1.0 + np.exp(-2.0 * p * (3.0 + rtOff)))
        f4 = 1.0 / (1.0 + np.exp(-2.0 * p * (4.0 + rtOff)))
        f8 = 1.0 / (1.0 + np.exp(-2.0 * p * (8.0 + rtOff)))
        return f0 - f1 + f2 - f3 + f4 + f8
    
    def table_up_s_tail_g(t, p, rtOff):
        tB = t % 8
        return mpath.table_up_s_tail_b(tB + globalVar.tau, p, rtOff)
    
    def cos_tail_b(t):
        if t <= 4.0:
            return np.cos(np.pi * t)
        else:
            return 1
        
    def cos_tail_g(t):
        tB = t % 8
        return mpath.cos_tail_b(tB)
    
    def d_cos_tail_b(t):
        if t <= 4.0:
            return np.pi * np.sin(np.pi * t)
        else:
            return 0
        
    def d_cos_tail_g(t):
        tB = t % 8
        return mpath.d_cos_tail_b(tB)
    
    def d_table_s_tail_b(t, p, rtOff):
        e0 = np.exp(-2.0 * p * (t - (0.0 + rtOff)))
        e1 = np.exp(-2.0 * p * (t - (1.0 + rtOff)))
        e2 = np.exp(-2.0 * p * (t - (2.0 + rtOff)))
        e3 = np.exp(-2.0 * p * (t - (3.0 + rtOff)))
        e4 = np.exp(-2.0 * p * (t - (4.0 + rtOff)))
        e8 = np.exp(-2.0 * p * (t - (8.0 + rtOff)))
        f0 = 2.0 * p * e0 / (1.0 + e0) ** 2
        f1 = 2.0 * p * e0 / (1.0 + e0) ** 2
        f2 = 2.0 * p * e0 / (1.0 + e0) ** 2
        f3 = 2.0 * p * e0 / (1.0 + e0) ** 2
        f4 = 2.0 * p * e0 / (1.0 + e0) ** 2
        f8 = 2.0 * p * e0 / (1.0 + e0) ** 2
        return -f0 + f1 - f2 + f3 - f4 + f8
        
    def d_table_s_tail_g(t, p, rtOff):
        tB = t % 8
        return mpath.d_table_s_tail_b(tB + globalVar.tau, p, rtOff)
    
    def table_s_tail_b(t, p, rtOff):
        f0 = 1.0 / (1.0 + np.exp(-2.0 * p * (t - (0.0 + rtOff))))
        f1 = 1.0 / (1.0 + np.exp(-2.0 * p * (t - (1.0 + rtOff))))
        f2 = 1.0 / (1.0 + np.exp(-2.0 * p * (t - (2.0 + rtOff))))
        f3 = 1.0 / (1.0 + np.exp(-2.0 * p * (t - (3.0 + rtOff))))
        f4 = 1.0 / (1.0 + np.exp(-2.0 * p * (t - (4.0 + rtOff))))
        f8 = 1.0 / (1.0 + np.exp(-2.0 * p * (t - (8.0 + rtOff))))
        return -f0 + f1 - f2 + f3 - f4 - f8
    
    def table_s_tail_g(t, p, rtOff):
        tB = t % 8
        return mpath.table_s_tail_b(tB + globalVar.tau, p, rtOff)
    
    def dtable_b(t, p, rtOff):
        e0 = np.exp(-2.0 * p * (t - (0.0 + rtOff)))
        e1 = np.exp(-2.0 * p * (t - (1.0 + rtOff)))
        e2 = np.exp(-2.0 * p * (t - (2.0 + rtOff)))
        e3 = np.exp(-2.0 * p * (t - (3.0 + rtOff)))
        e4 = np.exp(-2.0 * p * (t - (4.0 + rtOff)))
        f0 = 4.0 * p * e0 / (1.0 + e0) ** 2
        f1 = 4.0 * p * e1 / (1.0 + e1) ** 2
        f2 = 4.0 * p * e2 / (1.0 + e2) ** 2
        f3 = 4.0 * p * e3 / (1.0 + e3) ** 2
        f4 = 4.0 * p * e4 / (1.0 + e4) ** 2
        return -f0 + f1 - f2 + f3 - f4
    
    def dtable_g(t, p, rtOff):
        tB = t % 2
        return mpath.dtable_b(tB + globalVar.tau, p, rtOff)
    
    def table_b(t, p, rtOff):
        f0 = 2.0 / (1.0 + np.exp(-2.0 * p * (t - (0.0 + rtOff))))
        f1 = 2.0 / (1.0 + np.exp(-2.0 * p * (t - (1.0 + rtOff))))
        f2 = 2.0 / (1.0 + np.exp(-2.0 * p * (t - (2.0 + rtOff))))
        f3 = 2.0 / (1.0 + np.exp(-2.0 * p * (t - (3.0 + rtOff))))
        f4 = 2.0 / (1.0 + np.exp(-2.0 * p * (t - (4.0 + rtOff))))
        return 1.0 - f0 + f1 - f2 + f3 - f4
    
    def table_g(t, p, rtOff):
        tB = t % 2
        y = mpath.table_b(tB + globalVar.tau, p, rtOff)
        return y

### Airfoil M function

In [13]:
def airfoil_m(t, e, beta, gMax, p, rtOff, U, V):
    if (g.mpath == 0):
        l = -U * t + 0.5 * (np.cos(np.pi * (t + globalVar.tau)) + e) * np.cos(beta)
        h = -V * t + 0.5 * (np.cos(np.pi * (t + globalVar.tau)) + e) * np.sin(beta)
        dl = -U - 0.5 * np.pi * np.sin(np.pi * (t + globalVar.tau)) * np.cos(beta)
        dh = -V - 0.5 * np.pi * np.sin(np.pi * (t + globalVar.tau)) * np.sin(beta)
        gam = mpath.table_g(t, p, rtOff)
        gam = gMax * gam
        alp = 0.5 * np.pi - beta + gam
        dgam = mpath.dtable_g(t, p, rtOff)
        dalp = gMax * dgam
    elif (g.mpath == 1):
        dl = -U + 0.5 * mpath.d_cos_tail_g(t + globalVar.tau) * np.cos(beta)
        dh = -V + 0.5 * mpath.d_cos_tail_g2(t + globalVar.tau) * np.sin(beta)
        l = -U * t + 0.5 * mpath.cos_tail_g(t + globalVar.tau, e) * np.cos(beta)
        h = -V * t + 0.5 * mpath.cos_tail_g2(t + globalVar.tau, e) * np.sin(beta)
        gam = mpath.table_s_tail_g2(t, p, rtOff)
        gam = gMax * gam
        alp = 0.5 * np.pi - beta + gam
        dgam = mpath.d_table_s_tail_g2(t, p, rtOff)
        dalp = gMax * dgam
    elif (g.mpath == 2):
        # Translational Motion
        dl = -U * 0.5 * mpath.d_cos_up_tail_g2(t + globalVar.tau) * np.cos(beta)
        dh = -V + 0.5 * mpath.d_cos_up_tail_g2(t + globalVar.tau) * np.sin(beta)
        l = -U * t + 0.5 * mpath.cos_up_tail_g2(t + globalVar.tau, e) * np.cos(beta)
        h = -V * t + 0.5 * mpath.cos_up_tail_g2(t + globalVar.tau, e) * np.sin(beta)
        # Rotational Motion
        gam = mpath.table_up_s_tail_g2(t, p, rtOff)
        gam = gMax * gam
        alp = 0.5 * np.i - beta + gam
        dgam = mpath.d_table_up_s_tail_g2(t, p, rtOff)
        dalp = gMax * dgam
    elif (g.mpath == 3):
        # Translational Motion
        dl = -U * 0.5 * mpath.d_cos_tail_g(t + globalVar.tau) * np.cos(beta)
        dh = -V + 0.5 * mpath.d_cos_tail_g(t + globalVar.tau) * np.sin(beta)
        l = -U * t + 0.5 * mpath.cos_tail_g(t + globalVar.tau, e) * np.cos(beta)
        h = -V * t + 0.5 * mpath.cos_tail_g(t + globalVar.tau, e) * np.sin(beta)
        # Rotational Motion
        gam = mpath.table_s_tail_g(t, p, rtOff)
        gam = gMax * gam
        alp = 0.5 * np.i - beta + gam
        dgam = mpath.d_table_s_tail_g(t, p, rtOff)
        dalp = gMax * dgam
    elif (g.mpath == 4):
        # Translational Motion
        dl = -U * 0.5 * mpath.d_cos_up_tail_g(t + globalVar.tau) * np.cos(beta)
        dh = -V + 0.5 * mpath.d_cos_up_tail_g(t + globalVar.tau) * np.sin(beta)
        l = -U * t + 0.5 * mpath.cos_up_tail_g(t + globalVar.tau, e) * np.cos(beta)
        h = -V * t + 0.5 * mpath.cos_up_tail_g(t + globalVar.tau, e) * np.sin(beta)
        # Rotational Motion
        gam = mpath.table_up_s_tail_g(t, p, rtOff)
        gam = gMax * gam
        alp = 0.5 * np.i - beta + gam
        dgam = mpath.d_table_up_s_tail_g(t, p, rtOff)
        dalp = gMax * dgam
    return alp, l, h, dalp, dl, dh

## Wing Global

In [14]:
def wing_global(istep, t, a, alp, l, h, xv, yv, xc, yc, dfc, ZW, U, V):
    zt = l + 1j * h
    ZWt = ZW

    if istep != 1:
        ZWt = ZW - zt

    zv = xv + 1j * yv
    zc = xc + 1j * yc
    expmia = np.exp(-1j * alp)
    ZVt = (a + zv) * expmia
    ZCt = (a + zc) * expmia
    ZV = ZVt + zt
    ZC = ZCt + zt

    # Unit normal vector of the airfoil in the wing-fixed system
    denom = np.sqrt(1 + dfc ** 2)
    nx = -dfc / denom
    ny = 1.0 / denom
    nc = nx + 1j * ny
    # Unit normal vector of the airfoil in the global system
    NC = nc * expmia

    return NC, ZV, ZC, ZVt, ZCt, ZWt

## Airfoil V

In [15]:
def airfoil_v(ZC, ZCt, NC, t, dl, dh, dalp):
    V = (dl + 1j * dh) - 1j * dalp * ZCt
    VN = np.real(np.conj(V) * NC)
    return VN

## Velocity W2

In [16]:
def velocity_w2(m, ZC, NC, ZF, GAMAw, iGAMAw):
    eps = globalVar.eps
    ZF_c = ZF[0:iGAMAw]
    GAMAw_c = GAMAw[0:iGAMAw]

    r_ = np.subtract(np.expand_dims(ZC, 0).transpose(), ZF_c)
    r = np.abs(r_)
    GF = np.where(r < eps, 0.+0.j, (1.0 / r_))
    GF = GF * np.where(r < globalVar.delta, (r / globalVar.delta) ** 2, 1.)

    VNW = np.sum(GAMAw_c * np.imag(np.expand_dims(NC, 0).transpose()
                                   * GF) / (2.0 * np.pi), 1)

    return VNW

## Velocity

In [17]:
def vel_vortex_improved(GAM, z, z0):
    r = np.abs(np.subtract(np.reshape(z, (z.shape[0], 1)), z0))
    c = np.subtract(np.reshape(z, (z.shape[0], 1)), z0)
    v = 1j * np.divide(GAM, c, out=np.zeros_like(c),
                       where=c != 0) / (2.0 * np.pi)
    v = v * np.where(r < globalVar.delta, (r / globalVar.delta) ** 2, 1.0)
    v = np.conjugate(v)
    return v


def velocity(ZF, iGAMAf, GAMA, m, ZV, GAMAw, iGAMAw):
    v1 = np.sum(vel_vortex_improved(GAMA[0:m], ZF[0:iGAMAf], ZV[0:m]), axis=1)
    v2 = np.sum(vel_vortex_improved(
        GAMAw[0:iGAMAw], ZF[0:iGAMAf], ZF[0:iGAMAw]), axis=1)
    vs1 = v1.shape[0]
    vs2 = v2.shape[0]

    v1_final = np.pad(v1, (0, max(vs2 - vs1, 0)), mode="constant")
    v2_final = np.pad(v2, (0, max(vs1 - vs2, 0)), mode="constant")

    return ((v1_final + v2_final) * -1)[0:iGAMAf]

# Input Variables
* Wing Geometry
    * **l_**: wing span (cm) (reduce by half to be used for 2d modeling)
    * **c_**: chord length (cm) (calculated while specifying airfoil shape)
    * **n**: # of data points that define the airfoil shape.
    * **m**: # of vortex points on the airfoil
    * **camber**: Camber (not specified yet) (0 is a straight airfoil)
* Wing Motion Parameters
    * **phiT_, phiB_**: stroke angles (degrees)
    * **a_**: rotation axis offset (cm)
    * **beta_**: stroke plane angle (degrees)
    * **f_**: flapping frequency (1/sec)
    * **gMax**: max rotation (degrees)
    * **p**: rotation speed parameter (nondimentional) $p >= 4$
    * **rtOff**: rotation timing offset (nondimentional)
        * $\text{rtOff}<0$: advanced, $\text{rtOff}=0$: symmetric, $\text{rtOff}>0$: delayed
* Fluid Parameters
    * **rho_**: air density, $g/cm^3$
    * **U_, V_**: ambient velocity ($cm/sec$, assume constant)
    * **itinc**: Time increment and # of time steps option 0 (manually sepcify), 1 (automatic)
    * Velocity Contour Plot Parameters:
        * **svInc**: space-fixed velocity plot increment
        * **svMax**: space-fixed velocity plot max velocity
        * **wvInc**: wing-fixed velocity plot increment
        * **wvMax**: wing-fixed velocity plot max velocity
* Time March Variables
    * **q**: Multiplier ($0 < q <= 1$)
    * **dt**: Change in time
    * **nstep**: Total number of steps

In [18]:
l_ = 0.5 * 5.0 # Change this number.
n = 250
atmp_ = 0.8
x_ = np.linspace(-atmp_, atmp_, n, endpoint=True)
camber = 0.0
y_ = camber * (atmp_ ** 2 - x_ ** 2)
c_ = x_[n - 1] - x_[0]
m = 50
phiT_ = 45
phiB_ = -45
a_ = 0
beta_ = -30
f_ = 30
gMax_ = 30
p = 5
rtOff = 0.0
rho_ = 0.001225
U_ = 100.0
V_ = 0.0
itinc = 1
svInc = 0.025
svMax = 2.5
g.svCont = np.arange(0.0, svMax + 1e-10, svInc)
wvInc = 0.1
wvMax = 7.0
g.wvCont = np.arange(0.0, wvMax + 1e-10, wvInc)
q = 1.0
dt = 0.025
nstep = 25

In [19]:
v_, t_, d_, e, c, x, y, a, beta, gMax, U, V = in_data(l_, phiT_, phiB_, c_, x_, y_, a_, beta_, f_, gMax_, U_, V_)

g.delta = 0.5 * c / (m - 1) * q

if itinc == 1:
    nperiod = 1
    dt = min(c / (m - 1), 0.1 * (4 / p))
    nstep = int(nperiod * np.ceil(2/dt))

air = np.sqrt(U_ ** 2 + V_ ** 2)
fk = 2 * f_ * d_ / air
r = 0.25 * (c_ / d_) * (p / t_) * (gMax / f_)
k = fk * r

if air <= 1e-03:
    r = 0.25 * (c_ / d_) * (p / t_) * (gMax / f_)

xv, yv, xc, yc, dfc, m  = mesh_r(c, x, y, n, m)

# Initialization Of Time March

* Initialize the wake vortex. (**GAMAw**)
    * GAMAw[0:2] step 1, GAMAw[2:4] step 2, ...
* Initialize teh free vortex magnitude array. (**GAMAf**)
    * This is the vortext to be shed or convected.
* Initialize the total wake vortex sum (**sGAMAw**)
* Initialize the total wake vortex number (**iGAMAw**)
* Initialize the # of vortices to be convected or shed. (**iGAMAf**)
* Initialize the free & wake vortex location array (before convection)(**ZF**)
    * ZF[0:2] step 1, ZF[2:4] step 2, ZF[4:6] step 3, ...
    * Leading edge: odd components
    * Trailing edge: even components
* Initialize the free & wake vortex location array (after convection)(**ZW**)
    * ZW[0:2] step 1, ZW[2:4] step 2, ZW[4:6] step 3, ...
    * Leading edge: odd components
    * Trailing edge: even components
* This is further transformed into a new body-fixed coordinate system.
* Linear and angular impulse arrays (**impulseLb, impulseAb, impulseLw, impulseLb**)

In [20]:
GAMAw = np.zeros(2 * nstep)
GAMAf = np.zeros(2 * nstep)
sGAMAw = 0.0
iGAMAw = 0
iGAMAf = 0
ZF = np.zeros(2 * nstep, dtype=complex)
ZW = np.zeros(2 * nstep, dtype=complex)
impulseLb = np.zeros(nstep, dtype=complex)
impulseLw = np.zeros(nstep, dtype=complex)
impulseAb = np.zeros(nstep)
impulseAw = np.zeros(nstep)
LDOT = np.zeros(nstep)
HDOT = np.zeros(nstep)

* Setup the matrix for the nonpenetration condition
* Use the wing-fixed coordinate system to calcuate the matrix coefficients.
* The matrix coefficients in the global system are identical to these and remain constant throughout the time steps.

In [21]:
MVN = matrix_coef(xv, yv, xc, yc, dfc, m)
MVN_lu = lu_factor(MVN)

ZETA = 0
if globalVar.vfplot == 1:
    if camber == 0.0:
        ZETA = c_mesh(c_, d_)
    else:
        ZETA = camber_mesh(c_, d_, camber)

# Time Marching

* Perform the time march using the values defined above for calculation.

In [22]:
%%time

iterations = []

for istep in range(nstep):
    
    step = {}
    step['istep'] = istep
    
    t = istep * dt

    alp, l, h, dalp, dl, dh = airfoil_m(t, e, beta, gMax, p, rtOff, U, V)

    LDOT[istep] = dl
    HDOT[istep] = dh

    NC, ZV, ZC, ZVt, ZCt, ZWt = wing_global(istep, t, a, alp, l, h, xv, yv, xc, yc, dfc, ZW, U, V)

    step['ZC'] = np.copy(ZC)
    step['NC'] = np.copy(NC)
    step['t'] = np.copy(t)
    
    VN = airfoil_v(ZC, ZCt, NC, t, dl, dh, dalp)
    VNW = velocity_w2(m, ZC, NC, ZF, GAMAw, iGAMAw)

    step['VN'] = np.copy(VN)

    GAMA = VN - VNW
    GAMA = np.append(GAMA, -sGAMAw)
    GAMA = lu_solve(MVN_lu, GAMA)
    
    step['iGAMAw'] = np.copy(iGAMAw)
    step['ZV'] = np.copy(ZV)
    step['ZW'] = np.copy(ZW)

    impulseLb[istep] = -1j * np.sum(GAMA * ZVt)
    impulseAb[istep] = 0.5 * np.sum(GAMA * np.abs(ZVt) ** 2)
    impulseLw[istep] = -1j * np.sum(GAMAw[0:iGAMAw] * ZWt[0:iGAMAw])
    impulseAw[istep] = 0.5 * np.sum(GAMAw[0:iGAMAw] * np.abs(ZWt[0:iGAMAw]) ** 2)
    
    step['GAMA'] = np.copy(GAMA)
    step['GAMAw'] = np.copy(GAMAw)
    step['U'] = np.copy(U)
    step['V'] = np.copy(V)
    step['alp'] = np.copy(alp)
    step['l'] = np.copy(l)
    step['h'] = np.copy(h)
    step['dalp'] = np.copy(dalp)
    step['dl'] = np.copy(dl)
    step['dh'] = np.copy(dh)

    iGAMAf = 2 * (istep + 1)

    ZF[iGAMAf - 2] = ZV[0]
    ZF[iGAMAf - 1] = ZV[m - 1]

    VELF = velocity(ZF, iGAMAf, GAMA, m, ZV, GAMAw, iGAMAw)

    ZW[0:iGAMAf] = ZF[0:iGAMAf] + VELF * dt

    iGAMAw = iGAMAw + 2
    GAMAw[iGAMAf - 2] = GAMA[0]
    GAMAw[iGAMAf - 1] = GAMA[m - 1]
    sGAMAw = sGAMAw + GAMA[0] + GAMA[m - 1]

    ZF = ZW
    
    iterations.append(step)

CPU times: total: 906 ms
Wall time: 918 ms


# Plots For Time March

## Plotting Functions

* `wing_global_plot`
* `air_foil_v`
* `plot_wake_vortex`
* `plot_velocity`
* `force_moment`
* `plot_vortex_m`

In [23]:
def wing_global_plot(ZC, NC, t):
    
    fig, ax = plt.subplots()
    
    ax.plot(np.real(ZC), np.imag(ZC), 'o')
    sf = 0.025
    xaif = np.real(ZC)
    yaif = np.imag(ZC)
    xtip = xaif + sf * np.real(NC)
    ytip = yaif + sf * np.imag(NC)
    ax.plot([xaif, xtip], [yaif, ytip])
    fig.savefig(f"{figuresDir}w2g_{np.round(t, 4)}.tif")
    ax.clear()
    plt.close(fig)
    

def air_foil_v_plot(ZC, NC, VN, t):
    
    fig, ax = plt.subplots()
    
    sf = 0.025
    xc = np.real(ZC)
    yc = np.imag(ZC)
    nx = np.real(NC)
    ny = np.imag(NC)
    xaif = xc
    yaif = yc
    xtip = xc + sf * VN * nx
    ytip = yc + sf * VN * ny
    ax.plot([xaif, xtip], [yaif, ytip])
    ax.axis('equal')
    ax.plot(xc, yc, 'o')
    fig.savefig(f"{figuresDir}AirfoilVg_{np.round(t, 4)}.tif")
    ax.clear()
    plt.close(fig)
    

def plot_wake_vortex(iGAMAw, ZV, ZW, istep):
    
    fig, ax = plt.subplots()
    
    xpltf = np.real(ZV)
    ypltf = np.imag(ZV)

    if istep == 0:
        ax.plot(xpltf, ypltf, '-k')
        fig.savefig(f"{figuresDir}wake/wake_{istep}.tif")
    else:
        xpltw = np.real(ZW)
        ypltw = np.imag(ZW)

        xpltwo = xpltw[1::2]
        ypltwo = ypltw[1::2]
        xpltwe = xpltw[::2]
        ypltwe = ypltw[::2]

        ax.plot(xpltf, ypltf, '-k',
                 xpltwo, ypltwo, 'ok',
                 xpltwe, ypltwe, 'or')
        fig.savefig(f"{figuresDir}wake/wake_{istep}.tif")

    ax.clear()
    plt.close(fig)
    

def igVELF(Z, ZV, ZW, GAMA, m, GAMAw, iGAMAw, U, V, alp, dalp, dl, dh):
    sz = np.size(Z)
    VV = complex(0, 0) * np.ones(sz)
    for J in range(1, m + 1):
        for i in range(1, sz[0] + 1):
            for j in range(1, sz[1] + 1):
                VV[i - 1, j - 1] = VV[i - 1, j - 1] + \
                    vel_vortex(GAMA[J - 1], Z[i - 1, j - 1], ZV[J - 1])
    for J in range(1, iGAMAw + 1):
        for i in range(1, sz[0] + 1):
            for j in range(1, sz[1] + 1):
                VV[i - 1, j - 1] = VV[i - 1, j - 1] + \
                    vel_vortex(GAMA[J - 1], Z[i - 1, j - 1], ZV[j - 1])
    return VV

def igVELOCITYF(Z, ZV, ZW, a, GAMA, m, GAMAw, iGAMAw, U, V, alp, dalp, dl, dh):
    sz = np.size(Z)
    VV = np.zeros(sz) + 1j * np.zeros(sz)
    Z_ = np.reshape(Z, (196, 1))
    VV = VV - np.sum((0.5 * 1j / np.pi) *
                     np.divide(GAMA, (np.subtract(Z_, ZV))), 1)
    VV = VV - np.sum((0.5 * 1j / np.pi) *
                     np.divide(GAMAw[0:iGAMAw], (np.subtract(Z_, ZW[0:iGAMAw]))), 1)
    VV = np.conj(VV)
    VVspace = VV
    VVspace = VV + np.exp(1j * alp) * (U + 1j * V) * np.ones(sz)
    return VVspace

def plot_velocity(istep, ZV, ZW, a, GAMA, m,
                  GAMAw, iGAMAw, U, V, alp, l, h, dalp,
                  dl, dh, ZETA, vpFreq, zavoid, ivCont):
    
    fig, ax = plt.subplots()
    
    XPLTF = np.real(ZV)
    YPLTF = np.imag(ZV)

    # Plot the velocity field, every vpFreq seps.
    if istep % vpFreq == 0:
        # Calculate the velocity field.
        ROT = np.exp(-1j * alp)
        RZETA = (ZETA + a) * ROT

        X = np.real(RZETA) + l
        Y = np.imag(RZETA) + h
        Z = X + 1j * Y

        if zavoid == 1:
            VVspace = igVELF(Z, ZV, ZW, GAMA, m, GAMAw,
                             iGAMAw, U, V, alp, dalp, dl, dh)
        else:
            VVspace = igVELOCITYF(Z, ZV, ZW, a, GAMA, m, GAMAw,
                                  iGAMAw, U, V, alp, dalp, dl, dh)

        # Plot the velocity field in the space-fixed system.

        U = np.real(VVspace)
        V = np.imag(VVspace)
        S = np.sqrt(U * U + V * V)
        S = np.reshape(
            S, (int(math.sqrt(S.shape[0])), int(math.sqrt(S.shape[0]))))

        ax.quiver(X, Y, U, V)
        # plt.plot(XPLTF, YPLTF, '-b')
        fig.savefig(f"{figuresDir}velocity/spaceVelocity_{istep}.png")
        ax.clear()
        
        if ivCont == 1:
            ax.contour(X, Y, S, globalVar.svCont)
            ax.contourf(X, Y, S, globalVar.svCont)
        else:
            ax.contour(X, Y, S)
            ax.contourf(X, Y, S)

        ax.plot(XPLTF, YPLTF, '-b', linewidth='4')
        fig.savefig(f"{figuresDir}velocity/spaceSpeed_{istep}.png")
        ax.clear()
        plt.close(fig)
        

def force_moment(rho_, v_, d_, nstep, dt, U, V, impulseAb, impulseAw, impulseLb, impulseLw, LDOT, HODT):
    
    fig, ax = plt.subplots()
    
    forceb = np.zeros((nstep)) + 1j * np.zeros((nstep))
    forcew = np.zeros((nstep)) + 1j * np.zeros((nstep))
    force = np.zeros((nstep)) + 1j * np.zeros((nstep))
    momentb = np.zeros((nstep))
    momentw = np.zeros((nstep))
    moment = np.zeros((nstep))

    impulseAb = np.real(impulseAb)
    impulseAw = np.real(impulseAw)

    # Reference values of force and moment
    f_ = rho_ * (v_ ** 2) * d_
    m_ = f_ * d_

    for IT in range(nstep):

        U0 = (LDOT[IT] - U) + 1j * (HDOT[IT] - V)
        U0_conj = np.conj(U0)

        if IT == 0:
            forceb[0] = (impulseLb[1] - impulseLb[0]) / dt
            forcew[0] = (impulseLw[1] - impulseLw[0]) / dt
            momentb[0] = (impulseAb[1] - impulseAb[0]) / dt
            momentw[0] = (impulseAw[1] - impulseAw[0]) / dt

        elif IT == (nstep - 1):
            forceb[IT] = 0.5 * (3.0 * impulseLb[IT] - 4.0 * impulseLb[IT - 1] + impulseLb[IT - 2]) / dt
            forcew[IT] = 0.5 * (3.0 * impulseLw[IT] - 4.0 * impulseLw[IT - 1] + impulseLw[IT - 2]) / dt
            momentb[IT] = 0.5 * (3.0 * impulseAb[IT] - 4.0 * impulseAb[IT - 1] + impulseAb[IT - 2]) / dt
            momentw[IT] = 0.5 * (3.0 * impulseAw[IT] - 4.0 * impulseAw[IT - 1] + impulseAw[IT - 2]) / dt

        else:
            forceb[IT] = 0.5 * (impulseLb[IT + 1] - impulseLb[IT - 1]) / dt
            forcew[IT] = 0.5 * (impulseLw[IT + 1] - impulseLw[IT - 1]) / dt
            momentb[IT] = 0.5 * (impulseAb[IT + 1] - impulseAb[IT - 1]) / dt
            momentw[IT] = 0.5 * (impulseAw[IT + 1] - impulseAw[IT - 1]) / dt

        momentb[IT] = momentb[IT] + np.imag(U0_conj * impulseLb[IT])
        momentw[IT] = momentw[IT] + np.imag(U0_conj * impulseLw[IT])

        # Total force and moment ( these are on the fluid )
        # The dimensional force & moment on the wing are obtained by reversing the sign.
        # and multiplying the referse quantities
        force[IT] = -f_ * (forceb[IT] + forcew[IT])
        moment[IT] = -m_ * (momentb[IT] + momentw[IT])

    ITa = np.linspace(1, nstep, nstep, endpoint=True)

    ax.plot(ITa, np.real(force), 'x-k')
    fig.savefig(f"{figuresDir}fx.tif")
    ax.clear()
    ax.plot(ITa, np.imag(force), '+-k')
    fig.savefig(f"{figuresDir}fy.tif")
    ax.clear()
    ax.plot(ITa, moment, 'o-r')
    fig.savefig(f"{figuresDir}m.tif")
    ax.clear()
    plt.close(fig)

    # Calculate the average forces and moment
    Fx = np.real(force)
    Fy = np.imag(force)
    Mz = moment
    Fx_avr = np.average(Fx)
    Fy_avr = np.average(Fy)
    Mz_avr = np.average(Mz)
    

def plot_m_vortex(v_, d_, GAMAw, nstep):
    
    fig, ax = plt.subplots()
    
    gama_ = v_ * d_

    # Dimensional alues of the circulation
    GAMAwo = gama_ * GAMAw[0::2]
    GAMAwe = gama_ * GAMAw[1::2]
    it = list(range(1, nstep + 1))

    ax.plot(it, GAMAwo, 'o-k', it, GAMAwe, 'o-r')
    fig.savefig(f"{figuresDir}GAMAw.tif")
    ax.clear()
    plt.close(fig)

In [24]:
_ = force_moment(rho_, v_, d_, nstep, dt, U, V, impulseAb, impulseAw, impulseLb, impulseLw, LDOT, HDOT)
_ = plot_m_vortex(v_, d_, GAMAw, nstep)

In [25]:
%%time

def plot_fn(iteration):
    istep = step['istep']
    ZC = step['ZC']
    NC = step['NC']
    t = step['t']
    VN = step['VN']
    iGAMAw = step['iGAMAw']
    ZV = step['ZV']
    ZW = step['ZW']
    GAMA = step['GAMA']
    GAMAw = step['GAMAw']
    U = step['U']
    V = step['V']
    alp = step['alp']
    l = step['l']
    h = step['h']
    dalp = step['dalp']
    dl = step['dl']
    dh = step['dh']

    wing_global_plot(ZC, NC, t)
    air_foil_v_plot(ZC, NC, VN, t)
    plot_wake_vortex(iGAMAw, ZV, ZW, istep)
    plot_velocity(istep, ZV, ZW, a, GAMA, m, GAMAw, iGAMAw, U, V, alp, l, h, dalp, dl, dh, ZETA, globalVar.vpFreq, 
                  globalVar.zavoid, globalVar.ivCont)

def runner(iterations):
    
    pool = ThreadPool(processes=4)
    for _ in tqdm(pool.imap_unordered(plot_fn, iterations), total=len(iterations), ncols=80):
        pass

runner(iterations)

100%|█████████████████████████████████████████| 217/217 [01:03<00:00,  3.43it/s]

CPU times: total: 1min 12s
Wall time: 1min 3s
